In [1]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.0/894.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [2]:
!pip install transformers pillow torchvision

In [3]:
!pip install -U pinecone-client langchain chromadb tiktoken sentence-transformers pypdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.3 MB/s eta

In [4]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 49.9 MB/s eta 0:00:00


In [5]:
!pip install nltk

# This is the Library we use for the program and try to import all the library, please run the install line first before import to the program below

In [19]:
import os
import re
import json
import anthropic
import numpy as np
import PyPDF2
import fitz
import nltk
import pymupdf
import torch
import time
import requests
import base64
import threading
import ipywidgets as widgets

In [7]:
from PIL import Image
#from openai import OpenAI
from io import BytesIO
from IPython.display import display
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pinecone import Pinecone, ServerlessSpec, PineconeException
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from torchvision.models import resnet50
from transformers import BlipProcessor, BlipForConditionalGeneration

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
# Use environment variables for API keys and being the backbone of the program
os.environ["ANTHROPIC_API_KEY"] = #Enter your Claude API Key here

# Set Pinecone API Key for the Database
pinecone_api_key = #Enter your Pinecone API Key here
pinecone_environment = "us-east-1"

client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"]) # Classify the client we use here then we do it later to confirm as a backup

try:
    pinecone_client = Pinecone(api_key=pinecone_api_key, environment=pinecone_environment) # Test whether Pinecone service is up or not and make it ready for the program
    print("Pinecone client initialized successfully")
except Exception as e:
    print(f"Error initializing Pinecone client: {str(e)}") # Fail to intialize due to database down, etc.

model = SentenceTransformer('paraphrase-MiniLM-L6-V2') # classify the Sentence transformer we run
nltk.download('punkt') #model to help grab the sentence needed

# print the key of the pinecone and ready to run
print(f"Pinecone API Key: {pinecone_api_key[:5]}...{pinecone_api_key[-5:]}")
print(f"Pinecone Environment: {pinecone_environment}")

Pinecone client initialized successfully


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Pinecone API Key: ae690...2f755
Pinecone Environment: us-east-1


In [9]:
# This section is where we will be checking what file path is input receiving and tries to open the file
def is_pdf(file_path):
    return file_path.lower().endswith('.pdf') 

def is_image(file_path):
    try:
        Image.open(file_path)
        return True
    except:
        return False

In [10]:
# Image processing via the Bot itself aka Claude
def encode_image(file_content, file_name): # This will input the information we got from user to encode the image as UTF-8 Unicode and then classify which file format is and is it support with Claude
    base64_image = base64.b64encode(file_content).decode('utf-8')
    file_extension = file_name.split('.')[-1].lower()
    media_type = f"image/{file_extension}"
    return base64_image, media_type

In [11]:
# This is the function that handles the entire pdf file process
def extract_text_from_pdf(pdf_path): #This will get the text from pdf to plain text
    doc = fitz.open(pdf_path)
    text = []
    print(f"PDF has {len(doc)} pages")
    for page_num, page in enumerate(doc):
        blocks = page.get_text("blocks") #extract the text we got from the block we can see
        page_text = "\n".join([block[4] for block in blocks if block[4].strip()]) # join each block of text to the page
        text.append(page_text) # Add the text we need correlate to the page text we got and ready for the upsert process
        print(f"Extracted {len(page_text)} characters from page {page_num + 1}") # Grab the amount of text we grab by character to the page we gpt

    print(f"Total pages processed: {len(text)}") # Tell how many text grab
    return text # Return the text that is splited for the upsert

#Tell program to make pinecone index and create as the specs below
def create_index_if_not_exists(index_name, dimension): # This will create new index to the desired configuration, we use cosine similatiry for this project
    try:
        if index_name not in pinecone_client.list_indexes():
            pinecone_client.create_index(
                name=index_name,
                dimension=dimension,
                metric='cosine',
                spec=ServerlessSpec(cloud="aws", region="us-east-1")
            )
            print(f"Index '{index_name}' created successfully.") # Tell user that the index is finished
        else:
            print(f"Index '{index_name}' already exists.") # When user already have the exact same naming for the index

        return pinecone_client.Index(index_name) # Return the pinecone index that we created
    except Exception as e:
        print(f"Error creating or accessing index: {str(e)}") # Flagged error due to various reason like variable call not properly
        return None

#Preparing the data as the chunk and ready to upload to database
def prep_data_for_upsert(pages, file_path, max_chunk_size=500): # Tell program how we going to create the text chunk
    all_chunks = [] # Variable to store the text chunk
    for page_num, text in enumerate(pages): # for each page that it able to extract
        page_chunks = [] # create the chunk for each page
        sentences = nltk.sent_tokenize(text)  # Use NLTK for sentence tokenization
        current_chunk = '' #Display the text that it able to extract
        for sentence in sentences:
            if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:  # +1 for space
                current_chunk += sentence + ' ' #Create space to separate into chunks
            else:
                if current_chunk:
                    page_chunks.append(current_chunk.strip()) #Add this text to the chunk
                current_chunk = sentence + ' '
        if current_chunk:
            page_chunks.append(current_chunk.strip()) # When it hits the final chunk, add this chunk to the page as well, 
        all_chunks.extend([(chunk, page_num) for chunk in page_chunks]) # Iterate through the each chunk receive for each page to all of them
        print(f"Created {len(page_chunks)} chunks from page {page_num + 1}")

    print(f"Total chunks created: {len(all_chunks)} from {len(pages)} pages") #Tell how many chunk it creates

    # This will encode all of the chunks to the vector base and then send the information needed to add to the vector and return the data
    embeddings = model.encode([chunk for chunk, _ in all_chunks])
    data_to_upsert = [
        (f'{file_path}_p{page_num}_c{i}', embedding.tolist(), {'text': chunk, 'source': file_path, 'page': page_num})
        for i, ((chunk, page_num), embedding) in enumerate(zip(all_chunks, embeddings))
    ]
    print(f"Prepared {len(data_to_upsert)} vectors for upsert")
    return data_to_upsert

In [12]:
#This is where program index the file and prepare to upload to the pinecone database
def index_pdf_files(index, pdf_files, index_name):
    for pdf_file in pdf_files:
        try:
            pages = extract_text_from_pdf(pdf_file) # Finds the page that it has for the documents
            data = prep_data_for_upsert(pages, pdf_file) #Get the text chunk ready for upsert to the index

            print(f"Prepared {len(data)} vectors for '{pdf_file}'") # Tell user that data is ready to upsert

            batch_size = 25  # Reduced batch size
            total_vectors = len(data) #Show how many list it needs to upload to the documents
            vectors_upserted = 0


            #This will try to upsert the information receive as the batch and tries to upserting with the retries maximum at 5 and tries slower to give Pinecone time to think
            for i in range(0, len(data), batch_size):
                batch = data[i:i+batch_size]
                max_retries = 5 # Retries need to make the code to upsert
                retry_delay = 1  # Initial retry delay in seconds

                for attempt in range(max_retries):
                    try:
                        upsert_response = index.upsert(vectors=batch) # Upsert or send data to Pinecone
                        vectors_upserted += len(batch) # Says how many chunk or upserted to the database and update to every iteration
                        print(f"Successfully upserted batch {i//batch_size + 1}/{(total_vectors + batch_size - 1)//batch_size} for '{pdf_file}'")
                        print(f"Upsert response: {upsert_response}")
                        time.sleep(1)  # Small delay between batches
                        break
                    except PineconeException as pe:
                        if attempt < max_retries - 1:
                            print(f"Pinecone error upserting batch for '{pdf_file}'. Retrying... (Attempt {attempt + 1}/{max_retries})")
                            print(f"Error details: {str(pe)}")
                            time.sleep(retry_delay)
                            retry_delay *= 2  # Exponential backoff
                        else:
                            print(f"Failed to upsert batch for '{pdf_file}' after {max_retries} attempts: {str(pe)}")
                    except Exception as e:
                        print(f"Unexpected error upserting batch for '{pdf_file}': {str(e)}")
                        print("Continuing with the next batch...")
                        break

            print(f"Finished processing '{pdf_file}' into '{index_name}'") #Tell user that the info is up to the Pinecone
            print(f"Total vectors upserted: {vectors_upserted} out of {total_vectors}") # This will tell how much data they got for upserting

            # Verify the index status after processing each file
            try:
                index_stats = index.describe_index_stats()
                print(f"Total vectors in index after processing '{pdf_file}': {index_stats.total_vector_count}") # Show how many index that the documents got
            except Exception as e:
                print(f"Error getting index stats: {str(e)}") #It can't receive the information of the index due to can't create one or improperly create one

        except Exception as e:
            print(f"Error processing file '{pdf_file}': {str(e)}") # it can't responses due to the file complexity

    print("Finished indexing all PDF files.") # Shows that it able to check the information and create the vector it needed

#Grab the results needed from the documents sorting the order by KNN within 10 nearest member
def retrieve_relevant_info(index, user_input, top_k=10):
    try:
        if index is None:
            raise ValueError("Index is not initialized") #When Pinecone failed to create new index or use the previous index

        query_embedding = model.encode([user_input]).tolist() #This will convert the user prompt to the vector based format and then use to find the KNN
        results = index.query(
            vector=query_embedding[0], #Vector checking using the first one
            top_k=top_k,
            include_metadata=True
        )
        print(f"Raw query results: {results}") #Display the information that it sorted

        if not results['matches']:
            return "No matching documents found in the index." #Can't find relevant info on the docs

        #This will return the responses of the file using the result it got and display each value
        relevant_docs = [f"Page {result['metadata']['page']}:\n{result['metadata']['text']}" for result in results['matches'] if 'metadata' in result and 'text' in result['metadata']]

        if not relevant_docs:
            return "No relevant text found in matching documents." #Can't find relevant info on the docs

        return "\n\n---\n\n".join(relevant_docs) # Return users with the result
    except Exception as e: # This will only run when it can't get the document's information
        print(f"Error in retrieve_relevant_info: {str(e)}")
        return f"Error retrieving information: {str(e)}"

In [13]:
def check_file_type(file_path): #This which check for the file path that user uploads, it it is PDF, return yes, other wise check case by case
    if not os.path.exists(file_path):
        return "not_found"
    elif file_path.lower().endswith('.pdf'):
        return "pdf"
    elif is_image(file_path):
        return "image"
    else:
        return "unsupported"

def is_image(file_path): #If it checks and found that image opne the image and return yes
    try:
        Image.open(file_path)
        return True
    except:
        return False

In [14]:
# Global variables
index = None
file_type = None
image_description = None
last_result = None
current_file_path = None

In [15]:
"""
This is the section where all the file is check whether it's PDF or Image or Non of the above

"""
def process_file(file_path, index_name='gdk-l1m-pdf', dimension=384):
    global index, file_type
    file_type = check_file_type(file_path)

    if file_type == "not_found":
        return "not_found", None, None
    elif file_type == "unsupported":
        return "unsupported", None, None
    elif file_type == "pdf":
        try:
            # Create index if it doesn't exist
            index = create_index_if_not_exists(index_name, dimension)

            # Process PDF
            pages = extract_text_from_pdf(file_path)
            data = prep_data_for_upsert(pages, file_path)

            # Index the PDF file
            index_pdf_files(index, [file_path], index_name)

            return "pdf", index, None
        except Exception as e:
            print(f"Error processing PDF file: {str(e)}")
            return "error", None, None
    elif file_type == "image":
        return "image", None, None

    return "unknown", None, None

In [16]:
"""

This will create the response needed according of what file type that user inputs

"""
def process_pdf_query(query, client): #PDF path
    try:
        if index is None:
            return "Error: PDF index not properly initialized. Please upload a PDF file first." #This refers to No Pinecone Index 

        print(f"Querying index with: {query}") #Showing what is user asking
        relevant_info = retrieve_relevant_info(index, query) #Retrieve information from what user says and find the nearest neighbor
        print(f"Retrieved info: {relevant_info}") #Show the data that user gets from the documents

        #This will check if user retrieve any informtion or not, if not it will look up the general information in websites, etc.
        if relevant_info:
            context = f"Relevant document information:\n{relevant_info}"
        else:
            context = "No relevant information found in the document. I'll try to answer based on general knowledge."

        # Construct the message correctly for Anthropic's API and send the information needed for the AI
        messages = [
            {
                "role": "user",
                "content": f"You are a helpful assistant. If no relevant information is found in the document, please state that clearly, and then try to answer based on your general knowledge. If you can't answer, suggest how the user might rephrase their query or what kind of document might contain the information they're looking for.\n\nContext: {context}\n\nQuestion: {query}"
            }
        ]

        #Create the respond that it needed to be generated and to be show as the final product
        response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=messages
        )
        return response.content #Show only the information needed to respond, aka the text part

    except Exception as e:
        return f"Error processing PDF query: {str(e)}. Please try uploading the file again or check if the file contains the information you're looking for." #This will trigger when it's too irrelevant or too board

def process_image_query(file_content, file_name, query, client): #Image Path
    try:
        #This will use to encode the image to UTF-8 unicode and will check for the file path as well to able to put into the API
        base64_image, media_type = encode_image(file_content, file_name) 

        #This will generate the responses needed for user when input the image
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"Describe this image in detail, and then answer the following question about it: {query}"
                        },
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": media_type, #They can handle JPG, PNG, or any widely known name
                                "data": base64_image #Send the UTF-8 data to the API
                            }
                        }
                    ]
                }
            ]
        )
        return response.content #Responses the text chunk

    except Exception as e:
        return f"Error processing image query: {str(e)}" #Fail to process the image due to various reason

In [26]:
def extract_plaintext_from_claude_response(response_content):
  """
  Extracts plain text from a Claude API response, handling potential formatting variations.

  Args:
      response_content (str): The content string from Claude's response.

  Returns:
      str: The extracted plain text.
  """

  try:
    if isinstance(response_content, dict) and 'text' in response_content:
        text = response_content['text']
    elif isinstance(response_content, str):
        text = response_content
    else:
        text = str(response_content)

    # Remove TextBlock wrappers
    text = re.sub(r"TextBlock\(text='(.*?)', type='text'\)", r"\1", text, flags=re.DOTALL)

    # Remove outer brackets
    text = text.strip('[]')

    # Replace \n with actual newlines
    text = text.replace('\\n', '\n')

    # Remove any remaining escape characters
    text = text.replace('\\', '')

    # Replace multiple newlines with a single newline
    text = re.sub(r'\n+', '\n', text)

    # Remove any remaining Markdown formatting
    text = re.sub(r"```(?:\w+\n)?(.*?)```", r"\1", text, flags=re.DOTALL)
    text = re.sub(r"\[(.*?)\]\((.*?)\)", r"\1", text)
    text = re.sub(r"[_*`]", "", text)

    # Trim leading/trailing whitespace from each line
    text = '\n'.join(line.strip() for line in text.split('\n'))

    # Remove any numbered list formatting
    text = re.sub(r'^\s*\d+\.\s*', '', text, flags=re.MULTILINE)

    return text.strip()

  except Exception as e:
      print(f"Error extracting plain text: {e}")
      return response_content  # Return original content if error occurs

In [27]:
# Create widgets
file_upload = widgets.FileUpload(description="Upload File:")
query_input = widgets.Text(description="Question:", placeholder="Enter your question here...")
submit_button = widgets.Button(description="Submit")
output = widgets.Output()

# Display widgets
display(file_upload, query_input, submit_button, output) #Make the widget to display

# Define submit function
def on_submit(b):
    global last_result, current_file_path, file_type, index #Use the globa var to help the return value
    with output:
        output.clear_output()
        if not file_upload.value and current_file_path is None:
            print("Please upload a file first.")    # This is when user didn't upload the file on first use
            return

        client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"]) # Get the API Key from Claude API

        if file_upload.value:
            file_content = next(iter(file_upload.value.values()))['content'] # Grab the content that user uploads the file with
            file_name = next(iter(file_upload.value.keys()))

            # Check for the file that user uploads
            print(f"Processing new file: {file_name}")
            file_extension = file_name.split('.')[-1].lower()

            #Check for file type/file extention
            if file_extension == 'pdf':
                file_type = "pdf"
            elif file_extension in ['jpg', 'jpeg', 'png']:
                file_type = "image"
            else:
                file_type = "unsupported"

            if file_type == "pdf":
                # Save the uploaded file temporarily for PDF processing
                with open(file_name, 'wb') as f:
                    f.write(file_content)
                current_file_path = file_name
                _, index, _ = process_file(current_file_path)
            elif file_type == "image":
                # For images, we don't need to save the file
                current_file_path = None
                index = None
            else:
                print(f"Unsupported file type: {file_extension}")
                return
        else:
            file_name = "previously uploaded file"

        print(f"Processing query for {file_name}...") # show file use for indexing or use to describe 
        print(f"Full query: {query_input.value}") # The query or the prompt that you use for the bot

        if file_type == 'pdf':
          if index is None:
              result = "Error: PDF index not properly initialized. Please upload the PDF file again." # When Pinecone index fail to create
          else:
              result = process_pdf_query(query_input.value, client) # Create the responses for the PDF File
        elif file_type == 'image':
            result = process_image_query(file_content, file_name, query_input.value, client) # Create the responses for image
        else:
            result = "Error: Unsupported file type or file not properly processed"

        # Extract the text from Claude to plain text and display the text
        plain_text_result = extract_plaintext_from_claude_response(result)
        print("Result:", plain_text_result)
        last_result = plain_text_result

        if file_upload.value and file_type == 'pdf':
            # Remove the temporary PDF file
            os.remove(current_file_path)
        file_upload.value.clear()

        print("\nReady for the next query. You can upload a new file or ask another question about the current file.\n\n")
        if last_result:
            print(last_result)

# Connect button to function
submit_button.on_click(on_submit)

# Add a function to continuously process queries
def continuous_query(change):
    if change['type'] == 'change' and change['new']:
        on_submit(None)

# Connect the query input to the continuous query function
query_input.observe(continuous_query, names='value')

# Main loop to keep the program running
def run_program():
    global last_result
    while True:
        time.sleep(1)  # Add a small delay to prevent excessive CPU usage
        if last_result: # If there's previous response
            with output:
                print("\nLast Result:", last_result) #show last result
                last_result = None  # Reset last_result after displaying

# Run the program in a separate thread to keep the notebook responsive
threading.Thread(target=run_program, daemon=True).start()

Processing query for previously uploaded file...
Full query: H
Querying index with: H
Raw query results: {'matches': [{'id': 'deflood copy.pdf_p8_c9',
              'metadata': {'page': 8.0,
                           'source': 'deflood copy.pdf',
                           'text': 'Proof of Concept, \n'
                                   '\n'
                                   'w/ Tech used\n'
                                   '\n'
                                   'Example of the Dataset:\n'
                                   'S.D. of the Detection:'},
              'score': 0.273282975,
              'values': []},
             {'id': 'deflood copy.pdf_p1_c1',
              'metadata': {'page': 1.0,
                           'source': 'deflood copy.pdf',
                           'text': 'Meet Our Team\n'
                                   '\n'
                                   'Goood\n'
                                   '\n'
                                   'CEO\n'
        